# Common functionality

In [ ]:
from time import sleep
import apsuite.commisslib.meas_fofb_sysid as fofbsysid
print(fofbsysid.__file__)
from multiprocessing import Pool, set_start_method
from resource import getrusage, RUSAGE_SELF

from matplotlib import pyplot as plt
import matplotlib.ticker as ticker
import matplotlib
import numpy as np
import h5py
from os import makedirs

from apsuite.commisslib.meas_fofb_sysid import FOFBSysIdAcq
from siriuspy.devices import EVG, InjCtrl, SOFB, Trigger

fpath = './'

# PRBS Acquisitions

In [ ]:
acq = FOFBSysIdAcq()
sofb = SOFB(SOFB.DEVICES.SI)
injctrl = InjCtrl()
trigs = [Trigger('BO-Glob:TI-Mags-Fams'), Trigger('BO-Glob:TI-Mags-Corrs')]

# instantiate PVs so they are already connected when we try to use them
injctrl['AccumState-Sel']
injctrl['AccumState-Sts']

In [ ]:
def init_acq(acq):
    # parameters to generate singular mode levels
    acq.params.svd_levels_regularize_matrix = True
    acq.params.svd_levels_reg_sinval_min = acq.devices['fofb'].singval_min
    acq.params.svd_levels_reg_tikhonov_const = acq.devices['fofb'].tikhonov_reg_const
    acq.params.svd_levels_bpmsx_enbllist = acq.devices['fofb'].bpmxenbl
    acq.params.svd_levels_bpmsy_enbllist = acq.devices['fofb'].bpmyenbl
    acq.params.svd_levels_ch_enbllist = acq.devices['fofb'].chenbl
    acq.params.svd_levels_cv_enbllist = acq.devices['fofb'].cvenbl
    acq.params.svd_levels_rf_enbllist = acq.devices['fofb'].rfenbl
    acq.params.svd_levels_respmat = acq.devices['fofb'].respmat
    
    acq.params.svd_levels_singmode_idx = 0

In [ ]:
init_acq(acq)

In [ ]:
injctrl.injmode = 2
injctrl.target_current = 100

## data type

In [ ]:
# # ensure sysid data is configured - 0: raw, 1: lamp, 2:sysid
# acq.devices['famsysid'].set_data_type(2)

In [ ]:
acq.devices['famsysid'].data_type

## PRBS function definitions

In [ ]:
def prepare_for_acq(rfenbl, loopgain):
    # disable BO magnets triggers
    for trig in trigs:
        trig.state = 0

    # change fofb matrix enable list and gains
    ## open loop
    acq.devices['fofb'].cmd_turn_off_loop_state()

    ## reset correctors current and acc
    acq.devices['fofb'].cmd_corr_set_current_zero()

    ## set enbl list
    acq.devices['fofb'].rfenbl = rfenbl
    acq.params.svd_levels_rf_enbllist = acq.devices['fofb'].rfenbl

    ## set gains
    acq.devices['fofb'].loop_gain_h = loopgain
    acq.devices['fofb'].loop_gain_v = loopgain

    ## close loop again
    acq.devices['fofb'].cmd_turn_on_loop_state()

    # open sofb loop
    sofb.cmd_turn_off_autocorr()

def current_acq(acq, i, corr, val):
    bpm = corr2bpm[corr]
    corridx = acq.devices['famsysid'].psnames.index(corr)
    bpmidx = acq.devices['famsysid'].bpmnames.index(bpm)

    acq.params.prbs_bpms_to_get_data = np.zeros(160, dtype=bool)
    acq.params.prbs_corrs_to_get_data = np.zeros(160, dtype=bool)
    acq.params.prbs_corrs_to_get_data[corridx] = True
    acq.params.prbs_bpms_to_get_data[bpmidx] = True

    if 'C3' in corr:
        val /= 2

    lvls = acq.get_levels_corrs_indiv_exc(corrname=corr, lvl0=-val)

    acq.params.prbs_fofbacc_enbl = True
    acq.params.prbs_fofbacc_lvl0 = lvls
    acq.prepare_fofbacc_prbs()

    rv = acq.acquire_data()
    acq.data['excitation_info'] = {'corr': corr}
    return rv

def bpm_acq(acq, i, mode, val):
    
    if np.all(acq.params.svd_levels_bpmsx_enbllist == acq.params.svd_levels_bpmsy_enbllist):
        acq.params.prbs_bpms_to_get_data = acq.params.svd_levels_bpmsx_enbllist
    else:
        print('Enabled BPMs x and y lists are different!!!')
        
    acq.params.prbs_corrs_to_get_data = np.ones(160, dtype=bool)
    acq.params.prbs_bpmpos_enbl = True
    acq.params.svd_levels_singmode_idx = mode
    
    lvlsx, lvlsy = acq.get_levels_bpms_from_svd(ampmax = acq.params.svd_levels_ampmax, lvl0 = -val)
    lvlsx, lvlsy = np.asarray(lvlsx, dtype=int), np.asarray(lvlsy, dtype=int)
    acq.params.prbs_bpmposx_lvl0 = lvlsx
    acq.params.prbs_bpmposy_lvl0 = lvlsy
    acq.prepare_bpms_prbs()

    rv = acq.acquire_data()
    acq.data['excitation_info'] = {'mode': mode}
    return rv

def bpm_acq_rfprofile(acq, i, dev, val):
    
    if np.all(acq.params.svd_levels_bpmsx_enbllist == acq.params.svd_levels_bpmsy_enbllist):
        acq.params.prbs_bpms_to_get_data = acq.params.svd_levels_bpmsx_enbllist
    else:
        print('Enabled BPMs x and y lists are different!!!')

    acq.params.prbs_corrs_to_get_data = np.ones(160, dtype=bool)

    respmat = acq.devices['fofb'].respmat
    rfprof = respmat[:,-1].copy()
    rfprof /= np.linalg.norm(rfprof)
    rfprof *= val
    rfprof_x = np.array(rfprof[:160], dtype=int)
    rfprof_y = np.array(rfprof[160:], dtype=int)
    
    acq.params.prbs_bpmpos_enbl = True
    acq.params.svd_levels_singmode_idx = 0
    acq.params.prbs_bpmposx_lvl0 = -1 * rfprof_x
    acq.params.prbs_bpmposy_lvl0 = -1 * rfprof_y
    acq.prepare_bpms_prbs()

    rv = acq.acquire_data()
    acq.data['excitation_info'] = {'mode': 'rf profile'}
    return rv


def disable_prbs():
    print('DISABLING PRBS')
    # disable prbs
    acq.params.prbs_fofbacc_enbl = False
    acq.params.prbs_bpmpos_enbl = False
    zeroed = np.zeros(160)
    acq.params.prbs_fofbacc_lvl0 = zeroed
    acq.params.prbs_fofbacc_lvl1 = zeroed
    acq.params.prbs_bpmposx_lvl0 = zeroed
    acq.params.prbs_bpmposx_lvl1 = zeroed
    acq.params.prbs_bpmposy_lvl0 = zeroed
    acq.params.prbs_bpmposy_lvl1 = zeroed
    acq.prepare_fofbacc_prbs()
    acq.prepare_bpms_prbs()
    
    acq.trigger_timing_signal()
    
    # enable BO magnets triggres
    for trig in trigs:
        trig.state = 1
    
    # close sofb loop
    sofb.cmd_turn_on_autocorr()

## PRBS parameter configurations

In [ ]:
psnames = acq.devices['famsysid'].psnames
psnames = psnames[1:79] + psnames[81:-1]

modes = range(157)
acq.params.svd_levels_ampmax = 5000

current_prbs = {'name': 'current', 'lfsr': 7, 'sd': 3, 'taps': 1, 'val': 0.02, 'function': current_acq, 'items': psnames}
voltage_prbs = {'name': 'voltage', 'lfsr': 14, 'sd': 1, 'taps': 0, 'val': 1.107, 'function': current_acq, 'items': psnames}
#====== Varying gain ======#
bpm_prbs_rfdsbld_g1 = {'name': 'bpm-lfsr9-sd4-rfdsbld-0p052', 'lfsr': 9, 'sd': 4, 'taps': 0, 'val': 100, 'function': bpm_acq, 'items': modes, 'rfenbl': 0, 'loopgain': 0.052}
bpm_prbs_rfenbld_g1 = {'name': 'bpm-lfsr9-sd4-rfenbld-0p052', 'lfsr': 9, 'sd': 4, 'taps': 0, 'val': 100, 'function': bpm_acq, 'items': modes, 'rfenbl': 1, 'loopgain': 0.052}
bpm_prbs_rfdsbld_g2 = {'name': 'bpm-lfsr9-sd4-rfdsbld-0p100', 'lfsr': 9, 'sd': 4, 'taps': 0, 'val': 100, 'function': bpm_acq, 'items': modes, 'rfenbl': 0, 'loopgain': 0.100}
bpm_prbs_rfenbld_g2 = {'name': 'bpm-lfsr9-sd4-rfenbld-0p100', 'lfsr': 9, 'sd': 4, 'taps': 0, 'val': 100, 'function': bpm_acq, 'items': modes, 'rfenbl': 1, 'loopgain': 0.100}
bpm_prbs_rfdsbld_g3 = {'name': 'bpm-lfsr9-sd4-rfdsbld-0p150', 'lfsr': 9, 'sd': 4, 'taps': 0, 'val': 100, 'function': bpm_acq, 'items': modes, 'rfenbl': 0, 'loopgain': 0.150}
bpm_prbs_rfenbld_g3 = {'name': 'bpm-lfsr9-sd4-rfenbld-0p150', 'lfsr': 9, 'sd': 4, 'taps': 0, 'val': 100, 'function': bpm_acq, 'items': modes, 'rfenbl': 1, 'loopgain': 0.150}
#====== RF profile ======#
bpm_rfprof_prbs = {'name': 'bpm-lfsr9-sd4-rfprofile', 'lfsr': 9, 'sd': 4, 'taps': 0, 'val': 1000, 'function': bpm_acq_rfprofile, 'items': ['rfprofile', ], 'rfenbl': 0, 'loopgain': 0.052}

## Run PRBS acquisitions

In [ ]:
acq.params.acq_timeout = 120
acq.params.acq_nrpoints_before = 0
acq.params.acq_channel = 3  # sysid_applied
acq.params.acq_repeat = False
acq.params.acq_external = True

npoints = 50_000
exps = [bpm_prbs_rfdsbld_g1, bpm_prbs_rfenbld_g1, bpm_prbs_rfdsbld_g2, bpm_prbs_rfenbld_g2, bpm_prbs_rfdsbld_g3, bpm_prbs_rfenbld_g3]

for prbs in exps:
    print(prbs['name'])
    acq.params.prbs_lfsr_length = prbs['lfsr']
    acq.params.prbs_step_duration = prbs['sd']
    acq.params.prbs_mov_avg_taps = prbs['taps']
    acq.prepare_prbs()
    acq.sync_prbs()

    period = (2**prbs['lfsr'] - 1) * prbs['sd']
    acq.params.acq_nrpoints_after = np.floor(npoints / period) * period
    print(f'nrpoints_after: {acq.params.acq_nrpoints_after}')

    print(acq.devices['famsysid'].prbs_step_duration)
    print(acq.devices['famsysid'].prbs_lfsr_len)

    makedirs(fpath + prbs['name'], exist_ok = True)

    prepare_for_acq(prbs['rfenbl'], prbs['loopgain'])
    
    for i, dev in enumerate(prbs['items']):
        print(f'{prbs["name"]}: {i+1:03d}/{len(prbs["items"]):03d} -> {dev}')
        do_fn = False
        while not do_fn:
            do_fn = prbs['function'](acq, i, dev, prbs['val'])
        #break
        
        print('    saving data...')
        acq.fname = fpath + prbs['name'] + '/' + str(dev) + '.h5'
        acq.save_data(acq.fname)

        if i % 10 == 0:
            print('    injecting...')
            for trig in trigs:
                trig.state = 1
            sleep(2.5)
            injctrl['AccumState-Sel'] = 1
            sleep(1)
            while injctrl['AccumState-Sel']:
                sleep(.1)
            for trig in trigs:
                trig.state = 0
        print('    downloading FOFB...')
        sofb.cmd_turn_on_autocorr()
        sleep(5)
        sofb.cmd_turn_off_autocorr()
    else:
        disable_prbs()